# Introduction
## Purpose:
This file is meant to serve as a proof of concept for a project designed to estimate average salaries based on education and occupation, among other features, with data from the Bureau of Labor Statistics as its base. The immediate issue is that to prove that such a project is possible, we need to join several data sets together to create one that would best facilitate machine learning techniques. This notebook file is meant to figure that out using 2023 as an example year.

## Data Source:
(Note both of these is where I obtained data for 2023, I can get more years for Occupation Wages but I couldn't find more for education)
- Occupation Wage Data: https://www.bls.gov/mwe/tables.htm
- Education Data: https://www.bls.gov/emp/tables/unemployment-earnings-education.htm 

In [2]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns

## Downloading Education and Wage Data

In [4]:
edu_2023 = pd.read_excel('education-2023.xlsx', sheet_name='Table 5.4', header = 1)
edu_2023

,2023 National Employment Matrix title,2023 National Employment Matrix code,Typical education needed for entry,Work experience in a related occupation,Typical on-the-job training needed to attain competency in the occupation,xlsx_ooh_link
0,Chief executives,11-1011,Bachelor's degree,5 years or more,NaN,OOH Content
1,General and operations managers,11-1021,Bachelor's degree,5 years or more,NaN,OOH Content
2,Legislators,11-1031,Bachelor's degree,Less than 5 years,NaN,—
3,Advertising and promotions managers,11-2011,Bachelor's degree,Less than 5 years,NaN,OOH Content
4,Marketing managers,11-2021,Bachelor's degree,5 years or more,NaN,OOH Content
...,...,...,...,...,...,...
828,Wellhead pumpers,53-7073,High school diploma or equivalent,Less than 5 years,Moderate-term on-the-job training,—
829,Refuse and recyclable material collectors,53-7081,No formal educational credential,NaN,Short-term on-the-job training,OOH Content
830,"Tank car, truck, and ship loaders",53-7121,No formal educational credential,NaN,Short-term on-the-job training,—
831,"Material moving workers, all other",53-7199,No formal educational credential,NaN,Short-term on-the-job training,—


In [5]:
%%time 
wages_2023 = pd.read_excel('mwe-2023complete.xlsx', sheet_name ='MWE 2023', header=0)
wages_2023

CPU times: user 1min 4s, sys: 392 ms, total: 1min 5s
Wall time: 1min 5s


,Series Title,Area Level,Area Text,Occupation Text,Job Characteristic Text,Work Level Text,Average Hourly Wage ($),Average Hourly Wage\nFootnote,Relative standard \nerror,Relative standard error footnote,Series ID,Area code,Occupation \ncode,Job characteristic code,Work level code
0,Average hourly wage for level 06 management oc...,National,National,Management occupations,All workers,Level 06,19.51,NaN,-,2,WMU00000001020000001100000006,0,110000,0,06
1,Average hourly wage for level 07 management oc...,National,National,Management occupations,All workers,Level 07,27.51,NaN,-,2,WMU00000001020000001100000007,0,110000,0,07
2,Average hourly wage for level 08 management oc...,National,National,Management occupations,All workers,Level 08,34.55,NaN,-,2,WMU00000001020000001100000008,0,110000,0,08
3,Average hourly wage for level 09 management oc...,National,National,Management occupations,All workers,Level 09,42.15,NaN,-,2,WMU00000001020000001100000009,0,110000,0,09
4,Average hourly wage for level 10 management oc...,National,National,Management occupations,All workers,Level 10,46.66,NaN,-,2,WMU00000001020000001100000010,0,110000,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557748,Average hourly wage for part-time level 03 sto...,Nonmetro area,Eastern Wyoming nonmetropolitan area,Stockers and order fillers,Part-time,Level 03,18.06,NaN,-,2,WMU56000071020000005370652603,5600007,537065,26,03
557749,Average hourly wage for part-time not able to ...,Nonmetro area,Eastern Wyoming nonmetropolitan area,Stockers and order fillers,Part-time,Not able to be leveled,17.54,NaN,-,2,WMU56000071020000005370652616,5600007,537065,26,16
557750,Average hourly wage for part-time entry level ...,Nonmetro area,Eastern Wyoming nonmetropolitan area,Stockers and order fillers,Part-time,Entry,16.05,NaN,-,2,WMU560000710200000053706526A1,5600007,537065,26,A1
557751,Average hourly wage for part-time experienced ...,Nonmetro area,Eastern Wyoming nonmetropolitan area,Stockers and order fillers,Part-time,Experienced,18.06,NaN,-,2,WMU560000710200000053706526A3,5600007,537065,26,A3


### Observations:
It seems as though combining these datasets could happen either on the occupation code column for the wages data, which I believe is analogous to the National Employment Matrix code in the education data. Another possibility would be using the occupation text column from the wages data, which I believe is roughly analgous to the National Employment Matrix title column in the education data. This second option is less ideal as there is no guarantee that there will be exact occupation names that relate one to one between the two tables, and would likely require regex work to pull key words to relate an occupation from one table to the other. Thus, I believe it is more prudent to continue with the occupation code column, and doing processing to make them in the same form (extract the '-'), and seeing if this allows us for joining tables.

## Data Joining (Wages Data and Education Data)

### First: Preprocessing on the Occupation Codes

In [9]:
# First need to find out data types:
print('Occupation Code Data Type for Education Data:')
print(type(edu_2023['2023 National Employment Matrix code'][0]))
print('\n')

print('Occupation Code Data Type for Wages Data:')
print(type(wages_2023['Occupation \ncode'][0]))

Occupation Code Data Type for Education Data:
<class 'str'>


Occupation Code Data Type for Wages Data:
<class 'numpy.int64'>


In [10]:
# Next, remove characters in 2023 National Employment Matrix code from all non-null values:

edu_2023['2023 National Employment Matrix code'] = edu_2023['2023 National Employment Matrix code'].apply(
    lambda x: x.replace('-', '') if isinstance(x, str) else x
)

# Need to fill nulls:
edu_2023 = edu_2023.fillna(0)

# After that, cast the whole column to numpy.int64:
edu_2023 = edu_2023.astype({'2023 National Employment Matrix code' : np.int64})

In [11]:
edu_2023 = edu_2023.rename(columns={"2023 National Employment Matrix code": "Occupation Code"})
edu_2023['Occupation Code']

0      111011
1      111021
2      111031
3      112011
4      112021
        ...  
828    537073
829    537081
830    537121
831    537199
832         0
Name: Occupation Code, Length: 833, dtype: int64

In [12]:
wages_2023 = wages_2023.rename(columns={"Occupation \ncode": "Occupation Code"})
wages_2023['Occupation Code']

0         110000
1         110000
2         110000
3         110000
4         110000
           ...  
557748    537065
557749    537065
557750    537065
557751    537065
557752    537065
Name: Occupation Code, Length: 557753, dtype: int64

Preprocessing is complete. Time to attempt joining education and wage data based on occupation code.

### Second: Data Merging

In [15]:
# First obtain and check the occupation codes to identify potential discrepancies
edu_codes = edu_2023['Occupation Code'].unique()
wage_codes = wages_2023['Occupation Code'].unique()
in_wage_not_edu = []

for wage in wage_codes:
    if wage not in edu_codes:
        in_wage_not_edu.append(wage)

print('Current number of unaccounted wage occupation codes: ' + str(len(in_wage_not_edu)) + '\n' + 'Those values are:')
in_wage_not_edu

Current number of unaccounted wage occupation codes: 21
Those values are:


[110000,
 130000,
 150000,
 170000,
 190000,
 210000,
 230000,
 250000,
 270000,
 290000,
 310000,
 330000,
 350000,
 370000,
 390000,
 410000,
 430000,
 470000,
 490000,
 510000,
 530000]

In [16]:
# Merge the data with wages as base (left join) becuase we only need the wage codes that have equivalent education codes 
# (because those are the only ones we can do predictions with)
test_final = pd.merge(wages_2023, edu_2023, on='Occupation Code', how = 'left')

### Third: After Merge Postprocessing

In [18]:
# Clean up column names, remove unnecessary columns, count all nulls
test_final = test_final.rename(columns = {'Average Hourly Wage\nFootnote' : 'Average Hourly Wage Footnote', 'Relative standard \nerror' : 'Relative Standard Error'})
test_final = test_final.drop(['Series ID', 'xlsx_ooh_link', 'Average Hourly Wage Footnote', '2023 National Employment Matrix title', 'Relative Standard Error', 'Relative standard error footnote'], axis=1)
test_final.isnull().sum()

Series Title                                                                      0
Area Level                                                                        0
Area Text                                                                         0
Occupation Text                                                                   0
Job Characteristic Text                                                           0
Work Level Text                                                                   0
Average Hourly Wage ($)                                                           0
Area code                                                                         0
Occupation Code                                                                   0
Job characteristic code                                                           0
Work level code                                                                   0
Typical education needed for entry                                          

In [19]:
# We have enough data to drop nulls, especially as this is more of a proof of concept; drop nulls, and reorder columns
# Mostly alphabetically with Average Hourly Wage as final target column for machine learning 
final_data = test_final.dropna()
final_data = final_data.sort_index(axis=1)
new_cols = [col for col in final_data.columns if col != 'Average Hourly Wage ($)'] + ['Average Hourly Wage ($)']
final_data = final_data[new_cols]
final_data = final_data.reset_index(drop=True)
final_data

,Area Level,Area Text,Area code,Job Characteristic Text,Job characteristic code,Occupation Code,Occupation Text,Series Title,Typical education needed for entry,Typical on-the-job training needed to attain competency in the occupation,Work Level Text,Work experience in a related occupation,Work level code,Average Hourly Wage ($)
0,National,National,0,All workers,0,111011,Chief executives,Average hourly wage for not able to be leveled...,Bachelor's degree,0,Not able to be leveled,5 years or more,16,136.60
1,National,National,0,Nonunion,24,111011,Chief executives,Average hourly wage for nonunion chief executi...,Bachelor's degree,0,All levels,5 years or more,00,126.50
2,National,National,0,Full-time,25,111011,Chief executives,Average hourly wage for full-time chief execut...,Bachelor's degree,0,All levels,5 years or more,00,130.10
3,National,National,0,Time-based pay,27,111011,Chief executives,Average hourly wage for time-based chief execu...,Bachelor's degree,0,All levels,5 years or more,00,123.90
4,National,National,0,All workers,0,111021,General and operations managers,Average hourly wage for level 08 general and o...,Bachelor's degree,0,Level 08,5 years or more,08,32.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363976,Nonmetro area,Eastern Wyoming nonmetropolitan area,5600007,Part-time,26,537065,Stockers and order fillers,Average hourly wage for part-time level 03 sto...,High school diploma or equivalent,Short-term on-the-job training,Level 03,0,03,18.06
363977,Nonmetro area,Eastern Wyoming nonmetropolitan area,5600007,Part-time,26,537065,Stockers and order fillers,Average hourly wage for part-time not able to ...,High school diploma or equivalent,Short-term on-the-job training,Not able to be leveled,0,16,17.54
363978,Nonmetro area,Eastern Wyoming nonmetropolitan area,5600007,Part-time,26,537065,Stockers and order fillers,Average hourly wage for part-time entry level ...,High school diploma or equivalent,Short-term on-the-job training,Entry,0,A1,16.05
363979,Nonmetro area,Eastern Wyoming nonmetropolitan area,5600007,Part-time,26,537065,Stockers and order fillers,Average hourly wage for part-time experienced ...,High school diploma or equivalent,Short-term on-the-job training,Experienced,0,A3,18.06


### Observations
Overall, producing a dataframe of 363981 rows × 14 columns just for 2023 isn't the worst. Especially if it could be possible to incorporate 2020 - 2022 data as well, following the structure outlined up until now in terms of cleaning, pre and post processing. To round out this notebook as a proof of concept, I'm going to attempt some very basic preliminary predictions using machine learning techniques. I should probably just use the most basic versions of Linear Regression, RandomForest, and Gradient Boosting since that's what was mentioned in our original project proposal. I'll at least consider adding some basic cross validation, before potentially considering regularization and other techniques later, in the context of prying out the best performance we can get out of the data.

## Basic Model Generation

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [23]:
%%time
# Preprocessing for Linear Regression Model
X = final_data.drop('Average Hourly Wage ($)', axis=1)

# This column had two data types; was messing with the encoder so this line had to be added to address that
X['Typical on-the-job training needed to attain competency in the occupation'] = X['Typical on-the-job training needed to attain competency in the occupation'].apply(
    lambda x: 'None' if (x==0) else x
)

y = final_data['Average Hourly Wage ($)']

final_data = final_data.astype({'Area code': 'object', 'Job characteristic code': 'object', 'Occupation Code': 'object'})
categorical_features = X.columns
for col in categorical_features:
    X[col] = X[col].astype(str)

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Implementing Linear Regression Model:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_pipeline.fit(X_train, y_train)
predictions = model_pipeline.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)

# Output
print("R² Score:", r2_score(y_test, predictions))
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

R² Score: 0.9083569579042252
Mean Absolute Error (MAE): 2.7216696026992877
Root Mean Squared Error (RMSE): 4.275500390753642
CPU times: user 29.6 s, sys: 29.8 s, total: 59.3 s
Wall time: 11 s


In [24]:
%%time
# Random Forest Regressor pipeline
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=100,       
        max_depth=15,           
        n_jobs=-1,              
        random_state=42
    ))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

rf_pipeline.fit(X_train, y_train)
predictions = rf_pipeline.predict(X_test)

print("R² Score:", r2_score(y_test, predictions))
print("Random Forest MAE:", mean_absolute_error(y_test, predictions))
print("Random Forest RMSE:", mean_squared_error(y_test, predictions, squared=False))

R² Score: 0.8849519407724133
Random Forest MAE: 3.1563716026885342
Random Forest RMSE: 4.793305043066423
CPU times: user 31min 47s, sys: 11.5 s, total: 31min 59s
Wall time: 4min 40s


In [25]:
%%time
# Gradient Boosting Regressor pipeline
gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# Train and evaluate
gb_pipeline.fit(X_train, y_train)
predictions = gb_pipeline.predict(X_test)

print("R² Score:", r2_score(y_test, predictions))
print("Gradient Boosting MAE:", mean_absolute_error(y_test, predictions))
print("Gradient Boosting RMSE:", mean_squared_error(y_test, predictions, squared=False))

R² Score: 0.8283710439673454
Gradient Boosting MAE: 4.0038261771822885
Gradient Boosting RMSE: 5.854511994336896
CPU times: user 43.6 s, sys: 144 ms, total: 43.8 s
Wall time: 43.8 s
